# A Guided Tour of Ray Core: Remote Functions

[*Remote Functions*](https://docs.ray.io/en/latest/walkthrough.html#remote-functions-tasks)
involve using a `@ray.remote` decorator on a function. 

This implements a [*task parallelism*](https://patterns.eecs.berkeley.edu/?page_id=208) pattern, with properties: *data independence*, *stateless*

---

First, let's start Ray… 

This will start Ray on the local host, with headnode and workers for each core or CPU available.
You can check the resources being used.

In [ ]:
import logging
import ray

ray.init(
    ignore_reinit_error=True,              # Don't print error messages if a Ray instance is already running. Attach to it
    logging_level=logging.ERROR,           
)
ray.cluster_resources()                    # get the cluster resources

Ray Dashboard accessible at URI: [http://127.0.0.1:8265](http://127.0.0.1:8265)

## Remote Functions example

The following is just a regular Python function...

In [ ]:
def my_function ():
    return 42

When called, it simply returns an integer:

In [ ]:
my_function()

If you were to iterate through a sequence of calls to a function such as that, these calls would be performed *sequentially*.

However, by adding the `@ray.remote` decorator, a regular Python function becomes a Ray remote function:

In [ ]:
@ray.remote
def my_function ():
    return 42

To invoke this remote function, use the `remote` method. This will immediately return an object ref (a *future* in Python) and then create a task that will be executed on a worker process.

In [ ]:
%%time

obj_ref = my_function.remote()
obj_ref

The result can be retrieved with `ray.get`, which is a blocking call if the task is still not finished

In [ ]:
%%time

ray.get(obj_ref)

Invocations of Ray *remote functions* happen in parallel, and all computation gets performed in the background, driven by Ray's internal event loop. Remote calls return immediately, with a Python Future Object reference.

To illustrate this, first let's define a relatively "slow" function, by adding a 10 second delay...

In [ ]:
import time
import random

@ray.remote
def slow_function ():
  time.sleep(10)
  return random.randint(0, 9)

Now we'll iterate through multiple calls, showing that this does not block:

In [ ]:
%%time

futures_list = []

for i in range(4):
    future = slow_function.remote()
    futures_list.append(future)
print(futures_list)

In [ ]:
%%time

for future in futures_list:
    print(ray.get(future))

Note the difference between CPU times and wall clock?

## Another way to do this is using list comprehension

In [ ]:
%%time

futures_list = [slow_function.remote() for _ in range(4)]
futures_list

In [ ]:
%%time

values_list = [ray.get(future) for future in futures_list]
values_list

Note the difference between CPU times and wall clock in comprehensions? 
Comprehensions seems faster.

Finally, shutdown Ray

In [ ]:
ray.shutdown()

---
## References

[*Patterns for Parallel Programming*](https://www.goodreads.com/book/show/85053.Patterns_for_Parallel_Programming)  
Timothy G. Mattson, Beverly A. Sanders, Berna L. Massingill  
Addison-Wesley (2004)

[Ray Core Walkthrough](https://docs.ray.io/en/latest/walkthrough.html)
Ray Documentation and Gettting started materials